# Feature Data

In the previous notebook we have cleaned the main dataframe. In this notebook we go through the Civilian cases dataset and Facility Prisons, finally mergin them into one unique dataframe, that will be used in notebook 03 for data visualization and in 04, 05, 06 and 07 to modeling.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Civilian-Cases" data-toc-modified-id="Civilian-Cases-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Civilian Cases</a></span><ul class="toc-item"><li><span><a href="#Formatting" data-toc-modified-id="Formatting-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Formatting</a></span></li><li><span><a href="#Calculate-Montly-Data" data-toc-modified-id="Calculate-Montly-Data-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Calculate Montly Data</a></span></li><li><span><a href="#Export-Clean-Data" data-toc-modified-id="Export-Clean-Data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Export Clean Data</a></span></li></ul></li><li><span><a href="#Prison-Features" data-toc-modified-id="Prison-Features-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prison Features</a></span><ul class="toc-item"><li><span><a href="#Export-Cleaned-Dataset" data-toc-modified-id="Export-Cleaned-Dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Export Cleaned Dataset</a></span></li></ul></li><li><span><a href="#Merge-Datasets" data-toc-modified-id="Merge-Datasets-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Merge Datasets</a></span><ul class="toc-item"><li><span><a href="#Covid-Civilian-Cases" data-toc-modified-id="Covid-Civilian-Cases-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Covid Civilian Cases</a></span></li><li><span><a href="#Other-Prison-Features" data-toc-modified-id="Other-Prison-Features-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Other Prison Features</a></span></li><li><span><a href="#Export-Final-Dataset" data-toc-modified-id="Export-Final-Dataset-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Export Final Dataset</a></span></li></ul></li></ul></div>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import missingno as msno
import seaborn as sns

## Civilian Cases

The civilian cases dataset was colleted from Jhon Hopkins University.

In [3]:
civ_cases = pd.read_csv('./data/general_population_covid_cases_by_state.csv')

In [4]:
civ_cases.head()

,date,state,dataQualityGrade,deathIncrease (Deaths),positiveIncrease (increasing cases),hospitalizedIncrease (Hospitalized),totalTestsViralIncrease (Tested)
0,3/6/20,AK,unknown,0,0,0,8
1,3/7/20,AK,unknown,0,0,0,4
2,3/8/20,AK,unknown,0,0,0,2
3,3/9/20,AK,unknown,0,0,1,9
4,3/10/20,AK,unknown,0,0,0,0


Data are recorded daily in this dataset, but many States are reporting with different timeframes. For this reason we decide to reduce all columns to monthly data.

In [5]:
civ_cases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19206 entries, 0 to 19205
Data columns (total 7 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   date                                 19205 non-null  object
 1   state                                19205 non-null  object
 2   dataQualityGrade                     19205 non-null  object
 3   deathIncrease (Deaths)               19206 non-null  int64 
 4   positiveIncrease (increasing cases)  19206 non-null  int64 
 5   hospitalizedIncrease (Hospitalized)  19206 non-null  int64 
 6   totalTestsViralIncrease (Tested)     19206 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.0+ MB


To begin we extract a `month` feature from `date`.

*Note: this was possible because data start in March 2020 and end in January 2021, so no overlap of manth happens within the timeframe*

In [6]:
civ_cases['date'] = pd.to_datetime(civ_cases['date'])
civ_cases['month'] = civ_cases['date'].dt.month

In [7]:
civ_cases = civ_cases.drop(civ_cases[(civ_cases['date'].dt.year == 2021) & (civ_cases['month'] > 1)].index)

Not needing the date anymoe, we drop it.

In [8]:
civ_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18814 entries, 0 to 19205
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   date                                 18813 non-null  datetime64[ns]
 1   state                                18813 non-null  object        
 2   dataQualityGrade                     18813 non-null  object        
 3   deathIncrease (Deaths)               18814 non-null  int64         
 4   positiveIncrease (increasing cases)  18814 non-null  int64         
 5   hospitalizedIncrease (Hospitalized)  18814 non-null  int64         
 6   totalTestsViralIncrease (Tested)     18814 non-null  int64         
 7   month                                18813 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 1.3+ MB


### Formatting

Column names in this dataframe appear to be very unpractical for data analysis purposes. We rename them using a dictionary.

In [9]:
#Rename Columns
civ_cases = civ_cases.rename(columns={"dataQualityGrade": "civ_data_quality", "deathIncrease (Deaths)": "civ_death", "positiveIncrease (increasing cases)": "civ_cases", 'hospitalizedIncrease (Hospitalized)':'civ_hosp', 'totalTestsViralIncrease (Tested)': 'civ_tests'}, errors="raise")

### Calculate Montly Data

As mentioned in above and in previous notebook we want to aggregate data on a monthly bases.

To do so we start by creating a list of columns of interest.

In [10]:
col_ls = ['civ_death', 'civ_cases', 'civ_hosp', 'civ_tests']

Then we create an empty dataframe called `month_civ_cases` and iterating through the column list we just created we record a value per month per state, by operating a groupby and summing values.

In [11]:
month_civ_cases = pd.DataFrame()

In [12]:
for col in col_ls:
    month_civ_cases[col] = civ_cases.groupby(['state', 'month'])[col].agg('sum')

In [13]:
month_civ_cases.head(20)

civ_death  civ_cases  civ_hosp  civ_tests
state month                                           
AK    1.0           56       7137       182     224575
      3.0            3        128        12       3713
      4.0            6        227        26      15406
      5.0            1        108        12      32576
      6.0            4        481        18      60490
      7.0            9       2049        83     120921
      8.0           14       2293        90     115524
      9.0           19       2603        74     108577
      10.0          26       7423       140     147000
      11.0          39      16011       270     407189
      12.0          85      14138       298     264354
AL    1.0         2861      98413      7675          0
      3.0           13        981         0          0
      4.0          256       6038       978          0
      5.0          362      10884       866          0
      6.0          319      20142       925          0
      7.0          630      49678      7752          0
      8.0          602      38335      3746          0
      9.0          358      28643      2990          0
      10.0         427      37584      3193          0

Verified that the dataframe resulted correctly from the operation we proceed exporting this as `.csv`.

### Export Clean Data

In [14]:
month_civ_cases.to_csv('./data/month_civ_covid_clean.csv', index =False)

## Prison Features

From the Bureau of Justistice Statistics we collect data from two different reports:
- Prisoners in 2019    NCJ 255115
- Census of State and Federal Correctional Facilities, 2005

From each we pulled data aggregated at State level, that were not redundant and consistently available throught all the States.

The result of this analysis was collected in the `features.csv` file that we proceed analyzing further within.

In [15]:
features = pd.read_csv('./data/features.csv')

In [16]:
features.head()

,Region,Design capacity 2000/a,Design capacity 2005/a,Percent of design capacity occupied 2000,Percent of design capacity occupied 2005,Rated capacity 2000,Rated capacity 2005,Percent of rated capacity occupied 2000,Percent of rated capacity occupied 2005,Total_18,...,Inmates participating in one or more work programs,Prison industries,Support services,Farming,Public works,Other work programs,No work programs,phone_rate_avg,Latitude,Longitude
0,Alabama,17693.0,12918.0,127.0,179.0,22158.0,22158.0,101.0,0.0,26841.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.50,32.75,-86.75
1,Alaska,3490.0,3101.0,93.0,134.0,3622.0,4129.0,90.0,100.0,4380.0,...,1334.0,3.0,17.0,4.0,9.0,3.0,1.0,3.75,64.00,-150.00
2,Arizona,30426.0,30591.0,101.0,107.0,29482.0,34013.0,105.0,97.0,42005.0,...,20194.0,10.0,15.0,5.0,12.0,4.0,2.0,4.80,34.50,-111.50
3,Arkansas,10026.0,13913.0,104.0,100.0,10026.0,14673.0,104.0,95.0,17799.0,...,10739.0,0.0,24.0,2.0,7.0,2.0,1.0,4.80,34.75,-92.50
4,California,97740.0,95562.0,167.0,178.0,167367.0,120625.0,98.0,141.0,128625.0,...,77199.0,28.0,44.0,7.0,34.0,29.0,15.0,2.03,37.25,-119.75


In [17]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 27 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Region                                              50 non-null     object 
 1   Design capacity 2000/a                              50 non-null     float64
 2   Design capacity 2005/a                              50 non-null     float64
 3   Percent of design capacity occupied 2000            50 non-null     float64
 4   Percent of design capacity occupied 2005            50 non-null     float64
 5   Rated capacity 2000                                 50 non-null     float64
 6   Rated capacity 2005                                 50 non-null     float64
 7   Percent of rated capacity occupied 2000             50 non-null     float64
 8   Percent of rated capacity occupied 2005             50 non-null     float64
 9   T

In [18]:
features.rename(columns={'Region':'name'}, inplace=True)
#to simplify the merge later

We proceed changing the name of `Region` column to `name` to simpllify merge with datasets later.

### Export Cleaned Dataset

In [20]:
features.to_csv('./data/features_clean.csv', index= False)

## Merge Datasets

We then want to merge all datasets cleaned so far into one unique dataframe.

To do so we read the dataframes.

In [21]:
df = pd.read_csv('./data/month_prison_covid_clean.csv')

In [22]:
civ = pd.read_csv('./data/month_civ_covid_clean.csv')

In [23]:
feat = pd.read_csv('./data/features_clean.csv')

In [24]:
print(df.shape), print(civ.shape), print(feat.shape)

(510, 14)
(623, 6)
(50, 27)


(None, None, None)

It has to be noted that all Facilities Features are not recurrent on a monthly bases as the COVID data.

Also Civilian data have also ecords for US Territories.

In [25]:
#sum of columns - the ones we will merge on
(14+6+27)-3

44

### Covid Civilian Cases

We begin by merging all covid dataframes.

In [26]:
df.head()

,name,state,staff_tests,total_staff_cases,staff_recovered,total_staff_deaths,prisoner_tests,total_prisoner_cases,prisoners_recovered,total_prisoner_deaths,as_of_date,year,month,region
0,Alabama,AL,0.0,11.0,0.0,0.0,61.0,4.0,0.0,1.0,2020-04-28,2020.0,4.0,South
1,Alabama,AL,0.0,42.0,0.0,0.0,74.0,7.0,0.0,0.0,2020-05-26,2020.0,5.0,South
2,Alabama,AL,0.0,111.0,74.0,1.0,177.0,57.0,10.0,6.0,2020-06-30,2020.0,6.0,South
3,Alabama,AL,0.0,125.0,82.0,1.0,407.0,131.0,96.0,8.0,2020-07-28,2020.0,7.0,South
4,Alabama,AL,0.0,62.0,110.0,0.0,990.0,137.0,190.0,4.0,2020-08-25,2020.0,8.0,South


We do a merge of both dataframes on both `state` and `month`

In [27]:
df_covid = df.merge(civ, how='left', on=['state', 'month'])

In [28]:
df_covid.head()

,name,state,staff_tests,total_staff_cases,staff_recovered,total_staff_deaths,prisoner_tests,total_prisoner_cases,prisoners_recovered,total_prisoner_deaths,as_of_date,year,month,region,civ_death,civ_cases,civ_hosp,civ_tests
0,Alabama,AL,0.0,11.0,0.0,0.0,61.0,4.0,0.0,1.0,2020-04-28,2020.0,4.0,South,256,6038,978,0
1,Alabama,AL,0.0,42.0,0.0,0.0,74.0,7.0,0.0,0.0,2020-05-26,2020.0,5.0,South,362,10884,866,0
2,Alabama,AL,0.0,111.0,74.0,1.0,177.0,57.0,10.0,6.0,2020-06-30,2020.0,6.0,South,319,20142,925,0
3,Alabama,AL,0.0,125.0,82.0,1.0,407.0,131.0,96.0,8.0,2020-07-28,2020.0,7.0,South,630,49678,7752,0
4,Alabama,AL,0.0,62.0,110.0,0.0,990.0,137.0,190.0,4.0,2020-08-25,2020.0,8.0,South,602,38335,3746,0


### Other Prison Features

Finally, we merge the dataframe just created with the Facility Features'one, by merging on `name`.

In [29]:
df_final = df_covid.merge(feat, how= 'left', on=['name'])

In [30]:
df_final.head()

,name,state,staff_tests,total_staff_cases,staff_recovered,total_staff_deaths,prisoner_tests,total_prisoner_cases,prisoners_recovered,total_prisoner_deaths,...,Inmates participating in one or more work programs,Prison industries,Support services,Farming,Public works,Other work programs,No work programs,phone_rate_avg,Latitude,Longitude
0,Alabama,AL,0.0,11.0,0.0,0.0,61.0,4.0,0.0,1.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.5,32.75,-86.75
1,Alabama,AL,0.0,42.0,0.0,0.0,74.0,7.0,0.0,0.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.5,32.75,-86.75
2,Alabama,AL,0.0,111.0,74.0,1.0,177.0,57.0,10.0,6.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.5,32.75,-86.75
3,Alabama,AL,0.0,125.0,82.0,1.0,407.0,131.0,96.0,8.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.5,32.75,-86.75
4,Alabama,AL,0.0,62.0,110.0,0.0,990.0,137.0,190.0,4.0,...,28.0,7.0,10.0,6.0,9.0,5.0,7.0,4.5,32.75,-86.75


### Export Final Dataset

We then export the final dataset.

In [31]:
df_final.to_csv('./data/data_clean_complete.csv', index= False)